pipeline start
define functions to get populations processed in different useful ways

* la idea es que reciban como input una lista de diccionarios. Cada
diccionario contiene los parametros necesarios para alinear los spikes a un evento en particular
* tambien puede recibir como input attributos que se desean eliminar del objeto neurona (se elimina la informacion pero no el atributo per se)

In [4]:
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path
from typing import Dict, List
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.structures.population_data import PopulationData
import pandas as pd
import glob

In [5]:
# Define parameters
params = [
    {
        "loc": "in",
        "event": "sample_on",
        "time_before": 500,
        "time_after": 1000,
        "select_block": 1,
    },
    {
        "loc": "in",
        "event": "test_on_1",
        "time_before": 500,
        "time_after": 600,
        "select_block": 1,
    },
    {
        "loc": "out",
        "event": "sample_on",
        "time_before": 500,
        "time_after": 1000,
        "select_block": 1,
    },
    {
        "loc": "out",
        "event": "test_on_1",
        "time_before": 500,
        "time_after": 600,
        "select_block": 1,
    }
]

In [6]:
def read_and_compute(path,params,rf_loc):
    neu=NeuronData.from_python_hdf5(path)
    neu=neu.get_neu_align(params=params,delete_att=['sp_samples'],rf_loc=rf_loc)
    return neu

In [ ]:
# paths
filepaths = {
    "lip": "D:/data/session_struct/lip/neurons/",
    "pfc": "",
    "v4": "",
}
path_list={}
for area in filepaths.keys():
    path = filepaths[area]
    neu_path = path + "*neu.h5"
    path_list[area] = glob.glob(neu_path)

In [8]:
rf_loc= pd.read_csv("//envau_cifs.intlocal.univ-amu.fr/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/activation_index/rf_loc_df_lip.csv")

In [9]:
population = Parallel(n_jobs=-1)(
    delayed(read_and_compute)(path,params,rf_loc) for path in tqdm(path_list['lip'])
)

100%|██████████| 530/530 [03:37<00:00,  2.44it/s]


In [10]:
aa = PopulationData(population)
aa.to_python_hdf5('population.h5')
aa = PopulationData.from_python_hdf5('population.h5')